In [1]:
import pandas as pd
import numpy as np
import cvxpy as cvx
import random
from scipy import stats
import sympy
import itertools
import os
pathn = os.getcwd()
import matplotlib.pyplot as plt
from sklearn import preprocessing
import itertools
import math


import matplotlib.pyplot as plt
from matplotlib import pyplot

In [2]:
##define log10 function 

def log(x):
    return math.log10(x)

In [3]:
## call the dataset

data_dm = pd.read_csv("~/microbial-ecology/datasets/Luddington/FlygutCFUsData.csv")

In [4]:
### list the species provided

sp_list_provided = ['LP', 'LB', 'AP', 'AT', 'AO']

In [5]:
## dataframe label list

sp_frac_list = [str(x)+" abund (CFU)" for x in sp_list_provided]
sp_frac_list

['LP abund (CFU)',
 'LB abund (CFU)',
 'AP abund (CFU)',
 'AT abund (CFU)',
 'AO abund (CFU)']

In [6]:
data_dm[sp_frac_list].head(2)

,LP abund (CFU),LB abund (CFU),AP abund (CFU),AT abund (CFU),AO abund (CFU)
0,313000.0,0.0,0.0,0.0,0.0
1,268000.0,0.0,0.0,0.0,0.0


In [7]:
data_dm.head(2)

,treatment,LP,LB,AP,AT,AO,LP abund (CFU),LB abund (CFU),AP abund (CFU),AT abund (CFU),AO abund (CFU),Total CFU
0,1,1,0,0,0,0,313000.0,0.0,0.0,0.0,0.0,313000.0
1,1,1,0,0,0,0,268000.0,0.0,0.0,0.0,0.0,268000.0


In [8]:
#### total CFU is already given as the last column but we will check that the values are correct

totcfu = pd.DataFrame(data_dm[sp_frac_list].T.sum()) 
totcfugiven = pd.DataFrame(data_dm[sp_frac_list].T.sum().values)
((totcfu == totcfugiven).sum()) == len(totcfu)

0    True
dtype: bool

In [9]:
ablist = ['LP abund (CFU)','LB abund (CFU)','AP abund (CFU)','AT abund (CFU)','AO abund (CFU)','Total CFU']

In [10]:
data_dm[ablist].head(2)

,LP abund (CFU),LB abund (CFU),AP abund (CFU),AT abund (CFU),AO abund (CFU),Total CFU
0,313000.0,0.0,0.0,0.0,0.0,313000.0
1,268000.0,0.0,0.0,0.0,0.0,268000.0


In [11]:
abun_only = data_dm[ablist]
zerodf = data_dm[data_dm["Total CFU"]==0]
newidx = (data_dm.index).difference(set(zerodf.index))

In [12]:
zerodf.head(2)

,treatment,LP,LB,AP,AT,AO,LP abund (CFU),LB abund (CFU),AP abund (CFU),AT abund (CFU),AO abund (CFU),Total CFU
96,3,0,0,1,0,0,0.0,0.0,0.0,0.0,0.0,0.0
98,3,0,0,1,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
abun_only = abun_only.T[newidx].T

In [14]:
data_dm.T[newidx].T[sp_list_provided].head(2)

,LP,LB,AP,AT,AO
0,1.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0


In [15]:
for j in range(len(abun_only)):
    for k in range(5):
        abun_only.iloc[j][k] = abun_only.iloc[j][k]/abun_only.iloc[j][5]

In [16]:
#data_dm.T[newidx].T["treatment"].values

In [17]:
abun_only["treatment"] = data_dm.T[newidx].T["treatment"].values

In [18]:
uniqreltr = abun_only["treatment"].unique()

In [19]:
abun_rel = [] 
for i in uniqreltr:
    abun_rel.append(abun_only[abun_only["treatment"]==i][sp_frac_list].median().values)

In [20]:
pa_rel = []
for i in uniqreltr:
    pa_rel.append(data_dm[data_dm["treatment"]==i][sp_list_provided].mean().values)

In [21]:
padf = pd.DataFrame(pa_rel)

In [55]:
#padf.to_pickle("5sp-Gould-pa-wells.pkl")

In [22]:
padfT = padf.T
widx = [] 
for i in range(2**5-1):
    summ = 0
    for j in range(5):
        summ = summ + padfT[i][j]*2**(4-j)
    widx.append(summ)
padf["well index"] = widx
padf2  = padf.sort_values(by="well index")

In [23]:
relf = pd.DataFrame(abun_rel)
relf["well"] = padf2.sort_index()["well index"].values

In [34]:
relf

,0,1,2,3,4,well
0,1.000000,0.000000,0.000000,0.000000,0.000000,16.0
1,0.000000,1.000000,0.000000,0.000000,0.000000,8.0
2,0.000000,0.000000,0.999645,0.000000,0.000000,4.0
3,0.000000,0.000000,0.000000,1.000000,0.000000,2.0
4,0.000000,0.000000,0.000000,0.000000,1.000000,1.0
5,0.585047,0.416168,0.000000,0.000000,0.000000,24.0
6,0.518928,0.000000,0.480919,0.000000,0.000000,20.0
7,0.602310,0.000000,0.000000,0.397690,0.000000,18.0
8,0.523483,0.000000,0.000000,0.000000,0.476517,17.0
9,0.000000,0.731654,0.268346,0.000000,0.000000,12.0


In [24]:
reldff = relf.sort_values(by="well")

In [25]:
reldff.index = np.arange(1,32,1)

In [26]:
relT = reldff.T

In [27]:
relT[0] = [0]*6
reldff2 = relT.T
reldfvf = reldff2.sort_values(by="well")
reldfvf.head(2)

,0,1,2,3,4,well
0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,1.0,1.0


In [37]:
pd.to_pickle(reldfvf,"Gould-etal-5species-relative-abundance-Feb23median.pkl")

In [36]:
reldfvf.T

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.565823,0.481724,0.585047,0.460602,0.629310,0.306829,0.544546,0.382065,0.394408,0.312236
1,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.661875,...,0.000000,0.000000,0.416168,0.081378,0.089236,0.102313,0.093841,0.131730,0.153901,0.089474
2,0.0,0.0,0.0,0.000000,0.999645,0.036586,0.000000,0.000000,0.0,0.000000,...,0.147336,0.000000,0.000000,0.000000,0.000000,0.000000,0.278419,0.027490,0.077994,0.035746
3,0.0,0.0,1.0,0.342138,0.000000,0.000000,0.997195,0.309941,0.0,0.000000,...,0.245690,0.121142,0.000000,0.000000,0.159322,0.117504,0.000000,0.000000,0.116031,0.134151
4,0.0,1.0,0.0,0.657399,0.000000,0.964206,0.000000,0.635662,0.0,0.338125,...,0.000000,0.331602,0.000000,0.402386,0.000000,0.292760,0.000000,0.262259,0.000000,0.300376
well,0.0,1.0,2.0,3.000000,4.000000,5.000000,6.000000,7.000000,8.0,9.000000,...,22.000000,23.000000,24.000000,25.000000,26.000000,27.000000,28.000000,29.000000,30.000000,31.000000


In [32]:
pd.read_pickle("~/microbial-ecology/datasets/Gould-etal-5species-relative-abundance-Oct22.pkl").T

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.547310,0.487686,0.583029,0.460204,0.611555,0.341066,0.501534,0.399471,0.423199,0.288252
1,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.00005,0.618442,...,0.000000,0.000000,0.416752,0.146803,0.170989,0.204289,0.213226,0.213804,0.235517,0.170460
2,0.0,0.000000,0.000000,0.000000,0.999022,0.113582,0.124319,0.028231,0.00000,0.000000,...,0.186767,0.016945,0.000000,0.000000,0.000000,0.000000,0.285044,0.075752,0.159024,0.057286
3,0.0,0.000000,0.999693,0.294391,0.000000,0.000000,0.875119,0.384330,0.00000,0.000000,...,0.266092,0.149805,0.000000,0.000000,0.217456,0.133738,0.000000,0.000000,0.182580,0.151329
4,0.0,0.999985,0.000000,0.705667,0.000000,0.886622,0.000000,0.587360,0.00000,0.381219,...,0.000000,0.345747,0.000000,0.393032,0.000000,0.320831,0.000000,0.311213,0.000000,0.332509
well,0.0,1.000000,2.000000,3.000000,4.000000,5.000000,6.000000,7.000000,8.00000,9.000000,...,22.000000,23.000000,24.000000,25.000000,26.000000,27.000000,28.000000,29.000000,30.000000,31.000000


In [92]:
data_dm[data_dm["treatment"]==6].mean()

treatment              6.000000
LP                     1.000000
LB                     1.000000
AP                     0.000000
AT                     0.000000
AO                     0.000000
LP abund (CFU)    213402.308333
LB abund (CFU)    201315.645833
AP abund (CFU)         0.000000
AT abund (CFU)         0.000000
AO abund (CFU)         0.000000
Total CFU         414921.666667
dtype: float64